In [ ]:
#@title Mount drive files
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
import cv2
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import Image, display
from tensorflow import keras
from tensorflow.keras.utils import array_to_img, img_to_array

In [ ]:
model = tf.keras.applications.ResNet101()
preprocess_input = keras.applications.resnet.preprocess_input
decode_predictions = keras.applications.resnet.decode_predictions

IMG_SIZE = (224, 224, 3)
NUM_CLASSES = 1000
LAST_CONV_LAYER_NAME = 'conv5_block3_out'

# data.iloc[8232] # pdoffmed
# data.iloc[10994] # pdonmed
# data[data['task_data'] == 'pdonmed']
# data[data['eeg_data'] == 'shampd1eeg'] # 3737
# data[data['eeg_data'] == 'stim7pd2eeg'] # 9244
# data[data['eeg_data'] == 'stim8pd1eeg'] # 14424

In [ ]:
path_pacs_dataset = "/path-to-pacs-dir/"
df = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
df.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
df = df.dropna(subset=["peak_time", "reaction_time"])
print(df.info())
print(df)

### Randomly selected data for Grad-CAM

*
11 - hcoff - sham;
6154 - hcoff - stim7;
10110 - hcoff - stim8;

*
17944 - pdoff - sham;
19838 - pdoff - stim7;
26315 - pdoff - stim8;

*
29282 - pdon - sham;
34586 - pdon - stim7;
39684 - pdon - stim8;

In [ ]:
pac = df["pac_values"].iloc[11]

In [ ]:
array = img_to_array(array_to_img(pac.reshape(60, 30, 1), scale=False).resize((224, 224)))
array = np.dstack([array] * 3)
array = np.expand_dims(array, axis=0)

# Grad-**CAM**

In [ ]:
def get_img_array(pac_w, size=(224, 224, 3)):

    array = img_to_array(array_to_img(pac_w.reshape(60, 30, 1), scale=False).resize((224, 224)))
    array = np.dstack([array] * 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(LAST_CONV_LAYER_NAME).output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array = preprocess_input(get_img_array(pac, size=IMG_SIZE))
model.layers[-1].activation = None
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=2)[0])

In [ ]:
def save_and_display_gradcam(pac_w, heatmap, cam_path="cam.jpg", alpha=0.4):

    img = img_to_array(array_to_img(pac_w.reshape(60, 30, 1), scale=False).resize((224, 224)))
    heatmap = np.uint8(255 * heatmap)

    jet = cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    data = img_to_array(array_to_img(superimposed_img, scale=False).resize((30, 60)))
    superimposed_img = array_to_img(superimposed_img)

    superimposed_img.save(cam_path)

    display(Image(cam_path))
    return data

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=260)

plt.matshow(heatmap)
plt.axis('off')
plt.show()

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=285)
m = save_and_display_gradcam(pac, heatmap)

In [ ]:
path_sm = "/path-to-results-saliency-maps/"
np.save(path_sm + "saliency_h1.npy", m) 
print(m.shape)

In [ ]:
x = np.load(path_sm + "saliency_h1.npy")
print(np.median(x))

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=1)
save_and_display_gradcam(pac, heatmap)

# Guided Grad-CAM

In [ ]:
@tf.custom_gradient
def guided_relu(x):
    def grad(dy):
        return tf.cast(dy > 0, "float32") * tf.cast(x > 0, "float32") * dy
    return tf.nn.relu(x), grad

class GuidedBackprop:
    def __init__(self, model):
        self.model = model
        self.gb_model = self.build_guided_model()

    def build_guided_model(self):
        # build a guided version of the model replace ReLU with guided ReLU
        gb_model = tf.keras.Model(
            self.model.inputs, self.model.output
        )
        layers = [
            layer for layer in gb_model.layers[1:] if hasattr(layer, "activation")
        ]
        for layer in layers:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guided_relu
        return gb_model

    def guided_backprop(self, image: np.ndarray, class_index: int):
        expected_output = tf.one_hot([class_index] * image.shape[0], NUM_CLASSES)
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            tape.watch(inputs)
            outputs = self.gb_model(inputs)
            loss = tf.keras.losses.categorical_crossentropy(
                expected_output, outputs)
        grads = tape.gradient(loss, inputs)[0]
        return grads

In [ ]:
model = tf.keras.applications.ResNet101()
gb = GuidedBackprop(model)

saliency_map = gb.guided_backprop(img_array, class_index=100).numpy()

# Normalize with mean 0 and std 1
saliency_map -= saliency_map.mean()
saliency_map /= saliency_map.std() + tf.keras.backend.epsilon()
# Change mean to 0.5 and std to 0.25
saliency_map *= 0.25
saliency_map += 0.5
# Clip values between 0 and 1
saliency_map = np.clip(saliency_map, 0, 1)
# Change values between 0 and 255
saliency_map *= (2 ** 8) - 1
saliency_map = saliency_map.astype(np.uint8)

plt.axis('off')
plt.imshow(saliency_map)

In [ ]:
gb = GuidedBackprop(model)

# Guided grad_cam is guided backpropogation with feature importance coming from grad-cam
saliency_map = gb.guided_backprop(img_array, class_index=260).numpy()
gradcam = cv2.resize(heatmap, (224, 224))
gradcam = np.clip(gradcam, 0, np.max(gradcam)) / np.max(gradcam)
guided_gradcam = saliency_map * np.repeat(gradcam[..., np.newaxis], 3, axis=2)

# Normalize
guided_gradcam -= guided_gradcam.mean()
guided_gradcam /= guided_gradcam.std() + tf.keras.backend.epsilon()
guided_gradcam *= 0.25
guided_gradcam += 0.5
guided_gradcam = np.clip(guided_gradcam, 0, 1)
guided_gradcam *= (2 ** 8) - 1
guided_gradcam = guided_gradcam.astype(np.uint8)

plt.axis('off')
plt.imshow(guided_gradcam)